# Feature selection example
## Extract audio features 
Use `pyAudioAnalysis` library to extract audio features for all WAV files in folders `audio\music` and `audio\speech`. Use a cache pkl file to store audio features so that they not computed every time this notebook is executed. 

In [23]:
import os
from pyAudioAnalysis import MidTermFeatures as mtf
from pyAudioAnalysis import audioTrainTest as at
import numpy as np
import pickle
import os.path

if os.path.isfile('data2.pkl'):
    with open('data2.pkl','rb') as f:
        mid_term_features_2 = pickle.load(f)
        wav_file_list2 = pickle.load(f)
else:
    with open('data2.pkl','wb') as f:
        dirs = ['audio/music', 'audio/speech']
        # extract features from directories of WAV files:
        f1, _, feature_names = mtf.directory_feature_extraction(dirs[0], 1, 1, 0.1, 0.1)
        f2, _, feature_names = mtf.directory_feature_extraction(dirs[1], 1, 1, 0.1, 0.1)
        mid_term_features = [f1, f2]
        # convert list of feature matrices to x, y format:
        x, y = at.features_to_matrix(mid_term_features)
        print(x.shape)
        print(y.shape)
        m = x.mean(axis=0)
        s = np.std(x, axis = 0)
        x_2 = (x - m) / s
        pickle.dump(x_2, f)
        pickle.dump(wav_file_list2, f)
print(x_2.shape)

Analyzing file 1 of 20: audio/music/m_20451_0.0_1.0.wav
Analyzing file 2 of 20: audio/music/m_20620Nutropic20Feat.20Stephane20Belmondo20-20We27ve20Got20It21_0.0_1.0.wav
Analyzing file 3 of 20: audio/music/m_21020Teknic20Old20Skool20-20Stop20Trying_0.0_1.0.wav
Analyzing file 4 of 20: audio/music/m_21072_0.0_1.0.wav
Analyzing file 5 of 20: audio/music/m_21881_0.0_1.0.wav
Analyzing file 6 of 20: audio/music/m_22342_0.0_1.0.wav
Analyzing file 7 of 20: audio/music/m_22750_0.0_1.0.wav
Analyzing file 8 of 20: audio/music/m_23016_0.0_1.0.wav
Analyzing file 9 of 20: audio/music/m_23139_0.0_1.0.wav
Analyzing file 10 of 20: audio/music/m_23144_0.0_1.0.wav
Analyzing file 11 of 20: audio/music/m_23150_0.0_1.0.wav
Analyzing file 12 of 20: audio/music/m_23161_0.0_1.0.wav
Analyzing file 13 of 20: audio/music/m_23516_0.0_1.0.wav
Analyzing file 14 of 20: audio/music/m_23822_0.0_1.0.wav
Analyzing file 15 of 20: audio/music/m_23852_0.0_1.0.wav
Analyzing file 16 of 20: audio/music/m_24031_0.0_1.0.wav
Analy

In [24]:
x.shape

(40, 138)

## Feature selection

In [27]:
from sklearn.feature_selection import SelectFromModel
# #Selecting the Best important features according to Logistic Regression using SelectFromModel
sfm_selector = SelectFromModel(estimator=LogisticRegression())
sfm_selector.fit(x, y)
sfm_selector.get_support()

/usr/local/lib/python3.9/site-packages/pyAudioAnalysis/../sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([ True, False,  True, False, False,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False,  True,  True,  True,  True, False,  True, False,
       False, False,  True,  True, False,  True, False, False, False,
       False, False,

In [33]:
from itertools import compress
list(compress(feature_names, sfm_selector.get_support().tolist()))


['zcr_mean',
 'energy_entropy_mean',
 'spectral_entropy_mean',
 'spectral_rolloff_mean',
 'mfcc_1_mean',
 'mfcc_2_mean',
 'mfcc_3_mean',
 'mfcc_4_mean',
 'mfcc_5_mean',
 'mfcc_6_mean',
 'mfcc_8_mean',
 'mfcc_10_mean',
 'mfcc_11_mean',
 'mfcc_12_mean',
 'mfcc_13_mean',
 'energy_entropy_std',
 'mfcc_1_std',
 'mfcc_2_std',
 'mfcc_3_std',
 'mfcc_4_std',
 'mfcc_5_std',
 'mfcc_6_std',
 'delta energy_entropy_std',
 'delta mfcc_1_std',
 'delta mfcc_2_std',
 'delta mfcc_3_std',
 'delta mfcc_4_std',
 'delta mfcc_6_std',
 'delta mfcc_10_std',
 'delta mfcc_11_std',
 'delta mfcc_13_std',
 'ratio']